In [7]:
import os
import pandas as pd
import numpy as np
import cv2

from glob import glob
from tqdm import tqdm

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
def read_image(path, size):
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (size,size))
    image = image / 255.0
    image = image.astype(np.float32)
    return image

In [3]:
path = "Dog_Breed_Ident/"
train_path = os.path.join(path, "train/*")
test_path = os.path.join(path, "test/*")
labels_path = os.path.join(path, "labels.csv")
    
labels_df = pd.read_csv(labels_path)
breed = labels_df["breed"].unique()
print("Number of Breed: ", len(breed))
    
breed2id = {name: i for i, name in enumerate(breed)}
id2breed = {i: name for i, name in enumerate(breed)}

ids = glob(train_path)
labels = []

for image_id in ids:
    image_id = image_id.split("/")[-1].split(".")[0]
    breed_name = list(labels_df[labels_df.id == image_id]["breed"])[0]
    breed_index = breed2id[breed_name]
    labels.append(breed_index)

ids = ids[:1000]
labels = labels[:1000]

train_x, valid_x = train_test_split(ids, test_size=0.2, random_state=42)
train_y, valid_y = train_test_split(labels, test_size=0.2, random_state=42)

Number of Breed:  120


In [4]:
model = tf.keras.models.load_model("model.h5")

In [12]:
for i, path in tqdm(enumerate(valid_x[:10])):
    image = read_image(path, 224)
    image = np.expand_dims(image, axis=0)
    pred = model.predict(image)[0]
    label_index = np.argmax(pred)
    breed_name = id2breed[label_index]
    
    ori_breed = id2breed[valid_y[i]]
    ori_image = cv2.imread(path, cv2.IMREAD_COLOR)
    
    ori_image = cv2.putText(ori_image, breed_name, (0, 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 1)
    ori_image = cv2.putText(ori_image, ori_breed, (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    
    cv2.imwrite(f"save/valid_{i}.png", ori_image)

10it [00:00, 10.12it/s]
